In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp postprocessing

<IPython.core.display.Javascript object>

# Postprocessing

The postprocessing procedure is very similar to preprocessing.

The only difference between a postprocessing step and a preprocessing step is that preprocessing works on `feature_` columns while postprocessing manipulates `prediction_` columns.

Therefore, we also inherit from `BaseProcessor` for postprocessing. The PostProcessor should take a `Dataset` as input and output a `Dataset` where either:
1. `prediction_` columns are manipulated or
2. A new prediction column is added with prefix `prediction_`.

In [ ]:
# hide
from nbdev.showdoc import *

<IPython.core.display.Javascript object>

In [ ]:
#export
import numpy as np
import pandas as pd
import scipy.stats as sp
from typeguard import typechecked
from rich import print as rich_print
from sklearn.preprocessing import MinMaxScaler

from numerai_blocks.preprocessing import BaseProcessor, display_processor_info
from numerai_blocks.dataset import Dataset

<IPython.core.display.Javascript object>

## 1. Ensembling

Multiple prediction results can be ensembled in multiple ways, but we provide the most common use cases here.

In [ ]:
#export
@typechecked
class MeanEnsembler(BaseProcessor):
    """ Take simple mean of multiple cols and store in new col. """
    def __init__(self, cols: list, final_col_name: str):
        super(MeanEnsembler, self).__init__()
        self.cols = cols
        self.final_col_name = final_col_name
        assert final_col_name.startswith("prediction"), f"final_col name should start with 'prediction'. Got {final_col_name}"

    @display_processor_info
    def transform(self, dataset: Dataset, *args, **kwargs) -> Dataset:
        dataset.dataf.loc[:, [self.cols]][self.final_col_name] = dataset.dataf.loc[:, self.cols].mean(axis=1)
        rich_print(f":stew: Ensembled '{self.cols}' with simple mean and saved in '{self.final_col_name}' :stew:")
        return Dataset(**dataset.__dict__)

<IPython.core.display.Javascript object>

## 2. Feature Neutralization

In [ ]:
#export
@typechecked
class FeatureNeutralizer(BaseProcessor):
    """ Feature """
    def __init__(self, feature_names: list,
                 pred_name: str = "prediction",
                 proportion=0.5):
        super(FeatureNeutralizer, self).__init__()
        self.proportion = proportion
        self.feature_names = feature_names
        self.pred_name = pred_name
        self.new_col_name = f"{self.pred_name}_neutralized_{self.proportion}"

    @display_processor_info
    def transform(self, dataset: Dataset, *args, **kwargs) -> Dataset:
        neutralized_preds = dataset.dataf.groupby("era").apply(lambda x: self.normalize_and_neutralize(x, [self.pred_name], self.feature_names))
        min_max_scaled_preds = MinMaxScaler().fit_transform(neutralized_preds)
        dataset.dataf.loc[:, self.new_col_name] = min_max_scaled_preds
        rich_print(f":robot: Neutralized [bold]'{self.pred_name}'[bold] with proportion [bold]'{self.proportion}'[/bold] :robot:")
        rich_print(f"New neutralized column = [bold green]'{self.new_col_name}'[/bold green]")
        return Dataset(**dataset.__dict__)

    def _neutralize(self, scores, exposures):
        neutral_scores = scores - self.proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
        return neutral_scores / scores.std()

    @staticmethod
    def _normalize(dataf: pd.DataFrame):
        normalized_ranks = (dataf.rank(method="first") - 0.5) / len(dataf)
        return sp.norm.ppf(normalized_ranks)

    def normalize_and_neutralize(self, dataf: pd.DataFrame, pred_cols, by):
        # Convert the scores to a normal distribution
        preds, by_matrix = dataf[pred_cols], dataf[by].values
        preds = self._normalize(preds)
        preds = self._neutralize(preds, by_matrix)
        return preds

<IPython.core.display.Javascript object>

## 3. Custom PostProcessors

There are an almost unlimited number of ways to postprocess data. We invite the Numerai community to develop Numerai Classic and Signals preprocessors for `numerai-blocks`.

A new PostProcessor should inherit from `BaseProcessor` and implement a `transform` method. The `transform` method should take a `Dataset` as input and return a `Dataset` object as output. An example is given below.

We recommend adding `@typechecked` at the top of a new PostProcessor class to enforce types and provide useful debugging stacktraces.

To enable fancy logging output. Add the `@display_processor_info` decorator to the `transform` method.

Note that arbitrary metadata can be added or changed in the `Dataset` class during a postprocessing step.



In [ ]:
#export
@typechecked
class AwesomePostProcessor(BaseProcessor):
    """
    - TEMPLATE -
    Do some awesome postprocessing.
    """
    def __init__(self, *args, **kwargs):
        super(AwesomePostProcessor, self).__init__()

    @display_processor_info
    def transform(self, dataset: Dataset, *args, **kwargs) -> Dataset:
        # Do processing
        ...
        # Add new column for manipulated data (optional)
        new_column_name = "NEW_COLUMN_NAME"
        dataset.dataf.loc[:, f"prediction_{new_column_name}"] = ...
        ...
        # Parse all contents of Dataset to the next pipeline step
        return Dataset(**dataset.__dict__)

<IPython.core.display.Javascript object>